***Vorlesung 'Syntax natürlicher Sprachen'***

--- 
# Vorlesung 6: Intro Feature-Based Grammars


In [1]:
import nltk
from nltk import Tree

---
## 1. Feature-Modellierung von Subjekt-Kasus

### Feature-Grammar ohne Constraints (Variablen zum Hochreichen der Kasus-Information):

In [2]:
#Feature-Grammar ohne Constraints:
gramstring = r"""
% start S
    S   -> NP VP
    
    VP  -> VERBAL 
    VERBAL -> V NP
    VERBAL -> V

    NP[CASE=?x]  -> DET[CASE=?x] NOM
    NOM -> N    
    NOM -> N NP
 
    DET[CASE=NOM] -> "der"
    DET[CASE=AKK] -> "den"
    DET[CASE=DAT] -> "dem"    
    N   -> "Hund" | "Briefträger"
    V   -> "schläft" | "jagt" | "gehört"

"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=0)

In [3]:
#Negativbeispiel Subjektkasus:
sent = 'den Hund schläft'.split()
print(len(list(parser.parse(sent))))

for tree in parser.parse(sent):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)
    #Übergenerierung!

1
                                S[]          
                      ┌──────────┴──────┐     
                NP[CASE='AKK']         VP[]  
       ┌──────────────┴──────────┐      │     
       │                       NOM[] VERBAL[]
       │                         │      │     
DET[CASE='AKK']                 N[]    V[]   
       │                         │      │     
      den                       Hund schläft 



---
### Lösung Übergenerierung durch Subjekt-Kasus-Constraint:

In [4]:
gramstring = r"""
% start S

#Subjekt-Kasus-Constraint
    S   -> NP[CASE=NOM] VP
    
    VP  -> VERBAL 
    VERBAL -> V NP
    VERBAL -> V

    NP[CASE=?x]  -> DET[CASE=?x] NOM
    NOM -> N    
    NOM -> N NP
 
    DET[CASE=NOM] -> "der"
    DET[CASE=AKK] -> "den"
    DET[CASE=DAT] -> "dem"    
    N   -> "Hund" | "Briefträger"
    V   -> "schläft" | "jagt" | "gehört"
"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=0)

In [5]:
#Negativbeispiel Subjektkasus:
sent = 'den Hund schläft'.split()
print(len(list(parser.parse(sent))))

for tree in parser.parse(sent):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)
    #kein Parse

0


In [6]:
sent = 'der Hund schläft'.split()
print(len(list(parser.parse(sent))))

for tree in parser.parse(sent):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)

1
                                S[]          
                      ┌──────────┴──────┐     
                NP[CASE='NOM']         VP[]  
       ┌──────────────┴──────────┐      │     
       │                       NOM[] VERBAL[]
       │                         │      │     
DET[CASE='NOM']                 N[]    V[]   
       │                         │      │     
      der                       Hund schläft 



---
## 2. Feature-Modellierung von Objekt-Kasus

In [7]:
#Feature-Grammar ohne Constraints:
gramstring = r"""
% start S
    S   -> NP VP
    
    VP  -> VERBAL 
    VERBAL -> V NP
    VERBAL -> V

    NP[CASE=?x]  -> DET[CASE=?x] NOM
    NOM -> N    
    NOM -> N NP
 
    DET[CASE=NOM] -> "der"
    DET[CASE=AKK] -> "den"
    DET[CASE=DAT] -> "dem"    
    N   -> "Hund" | "Briefträger"
    V   -> "schläft" | "jagt" | "gehört"

"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=0)

In [8]:
#Negativbeispiel Objektkasus:
sent = 'der Hund jagt dem Briefträger'.split()
print(len(list(parser.parse(sent))))

for tree in parser.parse(sent):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)
    #Übergenerierung!

1
                                     S[]                                            
                      ┌───────────────┴──────────┐                                   
                      │                         VP[]                                
                      │                          │                                   
                      │                       VERBAL[]                              
                      │               ┌──────────┴──────────────┐                    
                NP[CASE='NOM']        │                   NP[CASE='DAT']            
       ┌──────────────┴──────────┐    │          ┌──────────────┴─────────────┐      
       │                       NOM[]  │          │                          NOM[]   
       │                         │    │          │                            │      
DET[CASE='NOM']                 N[]  V[]  DET[CASE='DAT']                    N[]    
       │                         │    │          │        

---
### Lösung Übergenerierung durch Objekt-Kasus-Constraint:

In [9]:
gramstring = r"""
% start S
    S   -> NP VP
    
    VP  -> VERBAL 
    VERBAL -> V[OBJCASE=?y] NP[CASE=?y]
    VERBAL -> V

    NP[CASE=?x]  -> DET[CASE=?x] NOM
    NOM -> N    
    NOM -> N NP
 
    DET[CASE=NOM] -> "der"
    DET[CASE=AKK] -> "den"
    DET[CASE=DAT] -> "dem"
    N   -> "Hund" | "Briefträger"
    V   -> "schläft"
    V[OBJCASE=AKK]   -> "jagt"
    V[OBJCASE=DAT]   -> "gehört"

"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=0)

In [10]:
#Negativbeispiel Objektkasus:
sent = 'der Hund jagt der Briefträger'.split()
print(len(list(parser.parse(sent))))

for tree in parser.parse(sent):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)
    #kein Parse

0


In [11]:
sent = 'der Hund jagt den Briefträger'.split()
print(len(list(parser.parse(sent))))

for tree in parser.parse(sent):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)

1
                                           S[]                                                  
                      ┌─────────────────────┴────────────────┐                                   
                      │                                     VP[]                                
                      │                                      │                                   
                      │                                   VERBAL[]                              
                      │                     ┌────────────────┴──────────────┐                    
                NP[CASE='NOM']              │                         NP[CASE='AKK']            
       ┌──────────────┴──────────┐          │                ┌──────────────┴─────────────┐      
       │                       NOM[]        │                │                          NOM[]   
       │                         │          │                │                            │      
DET[CASE='NOM']        

In [12]:
sent = 'der Hund gehört dem Briefträger'.split()
print(len(list(parser.parse(sent))))

for tree in parser.parse(sent):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)

1
                                           S[]                                                  
                      ┌─────────────────────┴────────────────┐                                   
                      │                                     VP[]                                
                      │                                      │                                   
                      │                                   VERBAL[]                              
                      │                     ┌────────────────┴──────────────┐                    
                NP[CASE='NOM']              │                         NP[CASE='DAT']            
       ┌──────────────┴──────────┐          │                ┌──────────────┴─────────────┐      
       │                       NOM[]        │                │                          NOM[]   
       │                         │          │                │                            │      
DET[CASE='NOM']        

---
## 3. Verhinderung Überproduktion durch invertierte NP - V -Regel für Auxiliarkonstruktion:

### Übergenerierung bei AUX-Konstruktion:

In [13]:
grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    
    VP  -> VERBAL 
    VERBAL -> V NP
    VP -> AUX VERBAL
 #invertierte NP-V-Regeln für AUX-VP:
    VERBAL -> NP V

    NP  -> DET NOM
    NOM -> N
 
    DET -> "der" | "den"
    N   -> "Hund" | "Briefträger"
    V   -> "übergibt"
    AUX -> "hat"
    V -> "übergeben"
""")

parser = nltk.ChartParser(grammar,trace=0)

In [14]:
#Negativbeispiel:
sent = 'der Briefträger hat übergeben den Hund'.split()
print(len(list(parser.parse(sent))))

for tree in parser.parse(sent):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)
    #Übergenerierung!

1
                     S                               
     ┌───────────────┴─────────────┐                  
     │                             VP                
     │               ┌─────────────┴────┐             
     │               │                VERBAL         
     │               │      ┌───────────┴─────┐       
     NP              │      │                 NP     
 ┌───┴───────┐       │      │           ┌─────┴───┐   
 │          NOM      │      │           │        NOM 
 │           │       │      │           │         │   
DET          N      AUX     V          DET        N  
 │           │       │      │           │         │   
der     Briefträger hat übergeben      den       Hund



--- 
### Lösung Übergenerierung mit binärem AUX-Feature:

In [15]:
gramstring = r"""
% start S

    S   -> NP VP
    
    VP  -> VERBAL 
    VERBAL -> V[-AUX] NP
    VP -> AUX VERBAL
    VERBAL -> NP V[+AUX]

    NP  -> DET NOM
    NOM -> N
 
    DET -> "der" | "den"
    N   -> "Hund" | "Briefträger"
    V[-AUX]   -> "übergibt"
    AUX -> "hat"
    V[+AUX] -> "übergeben"
"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=0)

In [16]:
#Negativbeispiel:
sent = 'der Briefträger hat übergeben den Hund'.split()
print(len(list(parser.parse(sent))))

for tree in parser.parse(sent):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)
    #kein Parse

0


In [17]:
sent = 'der Briefträger hat den Hund übergeben'.split()
print(len(list(parser.parse(sent))))

for tree in parser.parse(sent):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)

1
                        S[]                               
       ┌─────────────────┴──────────┐                      
       │                           VP[]                   
       │                 ┌──────────┴──────┐               
       │                 │              VERBAL[]          
       │                 │          ┌──────┴─────────┐     
      NP[]               │         NP[]              │    
  ┌────┴────────┐        │     ┌────┴──────┐         │     
  │           NOM[]      │     │         NOM[]       │    
  │             │        │     │           │         │     
DET[]          N[]     AUX[] DET[]        N[]     V[+AUX] 
  │             │        │     │           │         │     
 der       Briefträger  hat   den         Hund   übergeben



In [18]:
sent = 'der Briefträger übergibt den Hund'.split()
print(len(list(parser.parse(sent))))

for tree in parser.parse(sent):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)

1
                         S[]                       
       ┌──────────────────┴────────┐                
       │                          VP[]             
       │                           │                
       │                        VERBAL[]           
       │                  ┌────────┴──────┐         
      NP[]                │              NP[]      
  ┌────┴────────┐         │        ┌──────┴─────┐   
  │           NOM[]       │        │          NOM[]
  │             │         │        │            │   
DET[]          N[]     V[-AUX]   DET[]         N[] 
  │             │         │        │            │   
 der       Briefträger übergibt   den          Hund



In [19]:
#Negativbeispiel:
sent = 'der Briefträger den Hund übergibt'.split()
print(len(list(parser.parse(sent))))

for tree in parser.parse(sent):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)
    #kein Parse

0
